<a href="https://colab.research.google.com/github/Yasir-Ehtesham/Assignment-6-GenAI/blob/main/Assignment_6_Yasir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd


In [11]:
# Load the dataset
df = pd.read_csv("/Dataset__problem_1_1.csv").drop(['Unnamed: 0'],axis=1)
print(df.head())

                                                text  sentiment
0                                               text          0
1  rising cases of covid does not alarm me rising...          1
2  please vote for chicagoindiaresolution marking...          0
3  wishing all of you eidaladha hazrat ibrahim as...          1
4  daily coronavirus cases in india top for first...          1


In [12]:
# Check for null values
print("Checking for null values in the dataset:")
print(df.isnull().sum())

Checking for null values in the dataset:
text         18
sentiment     0
dtype: int64


In [13]:
# Drop rows with null values in 'tweet' or 'sentiment' columns
df.dropna(subset=['text', 'sentiment'], inplace=True)

# Verify that null values have been dropped
print("Null values after dropping:")
print(df.isnull().sum())


Null values after dropping:
text         0
sentiment    0
dtype: int64


In [14]:
import re

def preprocess_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from hashtags
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove special characters, numbers, punctuations
    tweet = re.sub(r'\d+','', tweet)
    tweet = re.sub(r'\W', ' ', tweet)
    tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet)
    tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', tweet)
    tweet = re.sub(r'\s+', ' ', tweet, flags=re.I)
    # Convert to lowercase
    tweet = tweet.lower()
    return tweet

df['cleaned_tweet'] = df['text'].apply(preprocess_tweet)
print(df['cleaned_tweet'].head())

0                                                 text
1    rising cases of covid does not alarm me rising...
2    please vote for chicagoindiaresolution marking...
3    wishing all of you eidaladha hazrat ibrahim as...
4    daily coronavirus cases in india top for first...
Name: cleaned_tweet, dtype: object


In [16]:
pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [17]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

In [18]:
# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

# For retrieval you need to pass this prompt.
query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [24]:
docs = df['cleaned_tweet']

docs.head()

docs_list = docs.tolist()

In [25]:
# 2. Encode
embeddings = model.encode(docs_list)

# Optional: Quantize the embeddings
binary_embeddings = quantize_embeddings(embeddings, precision="ubinary")

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)

similarities: tensor([[0.4062, 0.4376, 0.4300,  ..., 0.4130, 0.3620, 0.3715]])


In [28]:
df_1 = pd.DataFrame(embeddings)

In [29]:
df_1.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.116177,-0.578012,0.266820,0.612927,0.043412,-0.328882,0.070416,0.246055,0.953853,0.156534,...,-1.137794,0.624413,0.134859,-0.150926,-0.405047,-0.236336,0.117629,0.546044,0.229676,-0.145525
1,-0.438954,0.130028,-0.379822,-0.151033,-0.067663,-1.131480,0.457839,0.388525,0.212845,1.116551,...,-0.822510,-0.892992,-0.185816,-0.395896,-1.214439,0.361357,-0.603771,-0.201465,-0.107619,0.221382
2,-0.033647,-0.108035,-0.629190,-0.286074,-1.212927,-0.066357,0.221781,0.110041,0.152688,-0.247898,...,-1.132282,0.377680,-0.734767,0.292289,-0.779474,-0.408965,-0.695658,0.418231,1.239710,-0.860829
3,-0.044687,-0.018981,-0.232337,-0.060745,-0.691857,-0.488906,0.513378,0.910305,-0.048460,-0.488329,...,-0.724447,-0.046599,-0.239052,0.038681,-0.580674,0.137914,-0.748240,-0.635796,0.051760,-0.177839
4,-0.211173,-0.150212,-0.126380,0.003937,-0.817010,-0.478490,-0.128679,0.367624,0.503801,0.639812,...,-0.457523,0.263686,-0.249978,-0.253514,-0.793667,0.619124,-0.325835,-0.248071,-0.790655,-0.098776


In [30]:
df_1.shape

(134330, 512)

In [31]:
Sentiments = df["sentiment"]
df_1 = pd.concat([df_1,Sentiments], axis = 1)
df_1.head()

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,sentiment
0,-0.116177,-0.578012,0.266820,0.612927,0.043412,-0.328882,0.070416,0.246055,0.953853,0.156534,...,0.624413,0.134859,-0.150926,-0.405047,-0.236336,0.117629,0.546044,0.229676,-0.145525,0.0
1,-0.438954,0.130028,-0.379822,-0.151033,-0.067663,-1.131480,0.457839,0.388525,0.212845,1.116551,...,-0.892992,-0.185816,-0.395896,-1.214439,0.361357,-0.603771,-0.201465,-0.107619,0.221382,1.0
2,-0.033647,-0.108035,-0.629190,-0.286074,-1.212927,-0.066357,0.221781,0.110041,0.152688,-0.247898,...,0.377680,-0.734767,0.292289,-0.779474,-0.408965,-0.695658,0.418231,1.239710,-0.860829,0.0
3,-0.044687,-0.018981,-0.232337,-0.060745,-0.691857,-0.488906,0.513378,0.910305,-0.048460,-0.488329,...,-0.046599,-0.239052,0.038681,-0.580674,0.137914,-0.748240,-0.635796,0.051760,-0.177839,1.0
4,-0.211173,-0.150212,-0.126380,0.003937,-0.817010,-0.478490,-0.128679,0.367624,0.503801,0.639812,...,0.263686,-0.249978,-0.253514,-0.793667,0.619124,-0.325835,-0.248071,-0.790655,-0.098776,1.0


In [35]:
# Check for null values
print("Checking for null values in the dataset:")
print(df_1.isnull().sum())

Checking for null values in the dataset:
0            18
1            18
2            18
3            18
4            18
             ..
508          18
509          18
510          18
511          18
sentiment    18
Length: 513, dtype: int64


In [37]:
# Drop rows with null values in 'tweet' or 'sentiment' columns
df_1.dropna(inplace=True)

# Verify that null values have been dropped
print("Null values after dropping:")
print(df_1.isnull().sum())

Null values after dropping:
0            0
1            0
2            0
3            0
4            0
            ..
508          0
509          0
510          0
511          0
sentiment    0
Length: 513, dtype: int64


In [38]:
from sklearn.model_selection import train_test_split

X = df_1.drop('sentiment', axis=1)
y = df_1['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42, max_depth=20, min_samples_split=100, min_samples_leaf=100,
                             class_weight='balanced')
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=100, min_samples_split=100,
                       random_state=42)

In [40]:
from sklearn.metrics import classification_report

# Detailed classification report - Testing Data
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.57      0.59      0.58     13207
         1.0       0.59      0.56      0.58     13656

    accuracy                           0.58     26863
   macro avg       0.58      0.58      0.58     26863
weighted avg       0.58      0.58      0.58     26863



In [41]:
# Detailed classification report - Training Data
print(classification_report(y_train, clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.66      0.69      0.67     52438
         1.0       0.69      0.65      0.67     55011

    accuracy                           0.67    107449
   macro avg       0.67      0.67      0.67    107449
weighted avg       0.67      0.67      0.67    107449

